# An Array of Sequences

Before creating Python, Guido was a contributor to the ABC language—a 10-year research project to design a programming environment for beginners. ABC introduced many ideas we now consider “Pythonic”: generic operations on different types of sequences, built-in tuple and mapping types, structure by indentation, strong typing without variable declarations, and more. It’s no accident that Python is so user-friendly.

Python inherited from ABC the uniform handling of sequences. Strings, lists, byte sequences, arrays, XML elements, and database results share a rich set of common operations including iteration, slicing, sorting, and concatenation.

Most of the discussion in this chapter applies to sequences in general, from the familiar list to the str and bytes types that are new in Python 3. 

## Overview of Built-In Sequences

The standard library offers a rich selection of sequence types implemented in C:

- **Container sequences**
  - `list`, `tuple`, and `collections.deque` can hold items of different types, including nested containers.
- **Flat sequences**
  - `str`, `bytes`, `bytearray`, `memoryview`, and `array.array` hold items of one simple type.

**A container sequence holds references to the objects it contains**, which may be of any type, while **a flat sequence stores the value of its contents in its own memory space**, and not as distinct objects.

<img src="./images/flat-v-container.png" alt="flat-v-container" width=700 align="left" />

Thus, flat sequences are more compact, but they are limited to holding primitive machine values like `bytes`, `integers`, and `floats`.

> **Note:** Every Python object in memory has a header with metadata. The simplest Python object, a float, has two metadata fields. On a 64-bit Python build, the struct representing a float has these 64-bit fields: `*ob_refcnt`: the object’s reference count; `*ob_type`: a pointer to the object’s type; `*ob_fval`: a C double holding the value of the float. That’s why an ‍`array` with floats is much more compact than a `tuple` of floats: the `array` is a single object holding the raw values of the floats, while the tuple is several objects—the tuple itself and each float object contained in it.

Another way of grouping sequence types is by mutability:

- **Mutable sequences**
  - ‍‍`list`, ‍‍`bytearray`, ‍‍`array.array`, ‍‍`collections.deque`, ‍‍and `memoryvie`
- **Immutable sequences**
  - `tuple`, `str`, and `bytes`

> Keep in mind these common traits: **mutable** versus **immutable**; **container** versus **flat**. They are helpful to extrapolate what you know about one sequence type to others.

## Arrays

If a `list` will only contain numbers, an `array.array` is more efficient: it supports all mutable sequence operations (including `.pop`, `.insert`, and `.extend`), and additional methods for fast loading and saving such as `.frombytes` and `.tofile`.

A Python array is as lean as a C array. As shown in the previous figure, an array of float values does not hold full-fledged float instances, but only the packed bytes representing their machine values—similar to an array of double in the C language. When creating an array, you provide a `typecode`, a letter to determine the underlying C type used to store each item in the array. For example, `b` is the `typecode` for **signed char**. If you create an `array('b')`, then each item will be stored in a single byte and interpreted as an integer from –128 to 127. For large sequences of numbers, this saves a lot of memory. And Python will not let you put any number that does not match the type for the array.

Exmaple below shows creating, saving, and loading an array of 10 million floating-point random numbers.

In [38]:
# import the array type
from array import array
from random import random

In [39]:
#  Create an array of double-precision floats (typecode 'd')
# from any iterable object—in this case, a generator expression.
floats = array('d', (random() for i in range(10**7)))

In [ ]:
# Inspect the last number in the array.
floats[-1]

In [40]:
# Save the array to a binary file.
with open('floats.bin', 'wb') as f:
    floats.tofile(f)

In [46]:
# Create an empty array of doubles.
floats_2 = array('d')

In [47]:
# Read 10 million numbers from the binary file.
with open('floats.bin', 'rb') as f:
    floats_2.fromfile(f, 10**7)

In [44]:
# Inspect the last number in the array.
floats_2[-1]

0.03222361781776928

In [48]:
# Verify that the contents of the arrays match.
floats_2 == floats

True

As you can see, `array.tofile` and `array.fromfile` are easy to use. If you try the example, you’ll notice they are also very fast. A quick experiment shows that it takes about `0.1s` for `array.fromfile` to load 10 million double-precision floats from a binary file created with array.tofile. That is nearly 60 times faster than reading the numbers from a text file, which also involves parsing each line with the float built-in. Saving with `array.tofile` is about 7 times faster than writing one float per line in a text file. In addition, the size of the binary file with 10 million doubles is 80,000,000 bytes (8 bytes per double, zero overhead), while the text file has 181,515,739 bytes, for the same data.

> **Note:** Another fast—but more flexible—way of saving numeric data is the `pickle` module for object serialization. Saving an array of floats with `pickle.dump` is almost as fast as with `array.tofile`. However, `pickle` automatically handles almost all built-in types, including nested containers, and even instances of user-defined classes (if they are not too tricky in their implementation).

If you do a lot of work with arrays and don’t know about ‍‍`memoryview`, you’re missing out. See the next topic.

## Memory Views

The built-in `memoryview` class is a shared-memory sequence type that lets you handle slices of arrays without copying bytes. It was inspired by the NumPy library (which we’ll discuss shortly in “NumPy”). Travis Oliphant, lead author of NumPy, answers When should a memoryview be used? like this:

> A memoryview is essentially a generalized NumPy `array` structure in Python itself (without the math). It allows you to share memory between data-structures (things like PIL images, SQLite databases, NumPy arrays, etc.) without first copying. This is very important for large data sets.

Example below shows how to create alternate views on the same array of 6 bytes, to operate on it as 2×3 matrix or a 3×2 matrix:

In [49]:
from array import array

In [50]:
# Build array of 6 bytes (typecode 'B').
octets = array('B', range(6))

In [51]:
# Build memoryview from that array, then export it as list.
m1 = memoryview(octets)

In [52]:
m1.tolist()

[0, 1, 2, 3, 4, 5]

In [53]:
# Build new memoryview from that previous one, but with 2 rows and 3 columns.
m2 = m1.cast('B', [2, 3])

In [54]:
m2.tolist()

[[0, 1, 2], [3, 4, 5]]

In [55]:
# Yet another memoryview, now with 3 rows and 2 columns.
m3 = m1.cast('B', [3, 2])

In [56]:
m3.tolist()

[[0, 1], [2, 3], [4, 5]]

In [57]:
# Overwrite byte in m2 at row 1, column 1 with 22.
m2[1, 1] = 22

In [58]:
# Overwrite byte in m3 at row 1, column 1 with 33.
m3[1, 1] = 33

In [60]:
# Display original array, proving that the memory was shared among octets, m1, m2, and m3.
octets

array('B', [0, 1, 2, 33, 22, 5])

> **Note:** Indexing a memoryview using a `tuple`—as in the expression `m2[1, 1]` above—is a feature that was added in Python 3.5.